[주제] 테크 뉴스레터 작성하기

# CrewAI와 Tool 결합

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# 라이브러리 임포트

### crewai_tools
- SerperDevTool : Serper api를 활용한 결과물 받는 도구, 구글 웹검색 도구
- WebsiteSearchTool : 특정 키워드와 특정 웹사이트를 주면, 해당 웹 사이트에서 키워드와 관련이 있는 부부만 가져옴.
- ScrapeWebsiteTool : 어떤 URL을 주면 해당 url에 있는 텍스트를 스크래핑해서 가져오는 도구

In [3]:
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

# 필요한 객체 생성

In [4]:
# llm 객체
llm = ChatOpenAI(model="gpt-4o-mini")

# Tool
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

c:\Users\magpi\anaconda3\envs\agent_env\Lib\site-packages\alembic\config.py:577: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


# Agent 정의

In [5]:
# 테크 트렌드 연구원
researcher = Agent(
    role = "테크 트렌드 연구원",
    goal="인공지능 분야의 최신 기술 트렌드를 한국어로 제공합니다. 지금은 2025년 6월입니다.",
    backstory="기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.",
    tools=[search_tool, web_rag_tool],
    verbose=True,
    max_iter=5,  # 해당 Agent가 주어진 task를 수행할 때 반복 실행할 횟수를 제한하는 것임(웹검색을 5번 실행, 비용과 연결되어 있음.)
    llm=llm
)

- allow_delegation=False
    - 해당 agnet가 task를 수행할 때 혼자 힘으로 하기 힘들 때 다른 agnet에 위임하는 설정
    - detault=True
    - False로 하지 않으면 수행중에 researcher에게 작업을 넘기는 경우가 발생하는데 이것을 발생하지 않도록 제한하기 위함

In [6]:
# 뉴스레터 작성 agnet
writer = Agent(
    role="뉴스레터 작성자",
    goal="최신 AI 기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2025년 6월입니다.",
    backstory="기술에 대한 열정을 가진 숙련된 작가입니다.",
    verbose=True,
    allow_delegation=False,  
    llm=llm
)

# Task 정의

In [7]:
research = Task(
    description="AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.",
    expected_output="AI 업계에서 가장 주목받는 3대 기술 동향과 그 중요성에 대한 신선한 관점을 요약한 글",
    agent=researcher
)

In [8]:
write = Task(
    description="""
테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.
""",
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터", # 결과물 형식
    agent=writer,
    output_file=r"./new_post.md" # the final blog post will be saved here
)

# Crew 정의

In [9]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=False,
    process=Process.sequential,
    output_log_file="logs/crew_run.json",   # 👈 JSON 형태로 모든 스텝이 저장됩니다.
)

# Crew 실행

In [10]:
result = crew.kickoff()

# Agent: 테크 트렌드 연구원
## Task: AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.


# Agent: 테크 트렌드 연구원
## Thought: Action: Search the internet with Serper
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"2025 AI industry technology trends\"}"
## Tool Output: 
{'searchParameters': {'q': '2025 AI industry technology trends', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '5 AI Trends Shaping Innovation and ROI in 2025 | Morgan Stanley', 'link': 'https://www.morganstanley.com/insights/articles/ai-trends-reasoning-frontier-models-2025-tmt', 'snippet': "The world's biggest tech companies are vying to refine cutting-edge uses for artificial intelligence utilizations: large language models' ability to reason like humans; frontier models that push boundaries in natural-language processing, image generation, and coding; and the creation of systems that integrate ...", 'position': 1}, {'title': "6 AI trends you'll see more of in 2025 - Microsoft News", '

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\magpi\anaconda3\envs\agent_env\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]




# Agent: 테크 트렌드 연구원
## Thought: Action: Search in a specific website
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"AI technology trends 2025\", \"website\": \"https://www.morganstanley.com/insights/articles/ai-trends-reasoning-frontier-models-2025-tmt\"}"
## Tool Output: 
Relevant Content:
5 AI Trends Shaping Innovation and ROI in 2025 | Morgan Stanley Investment Banking Mar 20, 2025 AI’s Next Leap: 5 Trends Shaping Innovation and ROI Tweet this Share this on LinkedIn Share this on Facebook Email this Print this The world’s biggest tech companies presented at Morgan Stanley’s Technology, Media & Telecom Conference, identifying five trends around AI’s next frontiers and its ability to deliver ROI for enterprises. Key Takeaways In 2025, technology companies are focused on building AI platforms that meet their enterprise customers’ needs for optimized performance, profitability and security. In doing so, they’re partnering across the AI ecosystem of c

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\magpi\anaconda3\envs\agent_env\Lib\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]




# Agent: 테크 트렌드 연구원
## Thought: Action: Search in a specific website
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"AI technology trends 2025\", \"website\": \"https://www.technologyreview.com/2025/01/08/1109188/whats-next-for-ai-in-2025/\"}"
## Tool Output: 
Relevant Content:
5 AI Trends Shaping Innovation and ROI in 2025 | Morgan Stanley Investment Banking Mar 20, 2025 AI’s Next Leap: 5 Trends Shaping Innovation and ROI Tweet this Share this on LinkedIn Share this on Facebook Email this Print this The world’s biggest tech companies presented at Morgan Stanley’s Technology, Media & Telecom Conference, identifying five trends around AI’s next frontiers and its ability to deliver ROI for enterprises. Key Takeaways In 2025, technology companies are focused on building AI platforms that meet their enterprise customers’ needs for optimized performance, profitability and security. In doing so, they’re partnering across the AI ecosystem of chips companies

# 마크다운으로 표시

In [11]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# 최신 AI 기술 트렌드 뉴스레터 (2025년 6월호)

안녕하세요, 테크 팬 여러분! 2025년의 인공지능(AI) 세계는 정말 흥미진진한 변화로 가득 차 있습니다. 이번 뉴스레터에서는 AI 산업의 최신 동향을 세 가지 주요 트렌드를 통해 살펴보겠습니다. 자, 그럼 시작해볼까요?

## 1. 에이전트 기반 AI의 자율성 향상

최근 AI 기술이 눈부신 발전을 이루면서, 에이전트 기반 시스템은 더욱 더 많은 자율성을 갖추게 되었습니다. 이제 AI 에이전트는 사용자 요구를 더 정확하게 이해하고, 복잡한 문제를 해결하는 데 있어 마치 인간처럼 전략적으로 행동할 수 있는 능력을 갖추었습니다. 이런 변화는 기업들이 AI를 통해 생산성을 높이고, 운영 효율성을 극대화하는 데 큰 도움이 될 것입니다. 여러분의 업무를 도와줄 AI 비서가 곧 등장할지도 모르겠네요!

## 2. 대형 언어 모델의 진화

대형 언어 모델의 발전은 AI가 자연어 처리에서 인간과 비슷한 수준의 추론 능력을 갖추도록 하는 데 중점을 두고 있습니다. 이 모델은 텍스트, 이미지, 비디오 등 다양한 형태의 데이터를 통합하여 처리할 수 있는 능력을 가지게 되어, 멀티모달 데이터 처리의 새로운 지평을 열고 있습니다. 교육, 헬스케어, 고객 서비스 등 여러 분야에서 AI의 실질적 활용도가 높아질 것이며, 이는 우리의 일상생활을 더욱 풍부하게 만들어 줄 것입니다.

## 3. 클라우드 기반 AI와 커스텀 실리콘의 결합

AI 시스템의 성능을 극대화하기 위해 클라우드 서비스와 커스텀 하드웨어의 결합이 이루어지고 있습니다. 기업들은 클라우드 기반 AI 솔루션을 통해 자원을 효율적으로 관리하고 비용을 절감하며, 실시간 데이터 처리 능력을 강화할 수 있습니다. 이는 기업이 AI 기술을 보다 안전하고 효율적으로 도입할 수 있는 기반을 제공합니다. 앞으로 AI가 기업 전략의 핵심 요소로 자리매김하는 데 큰 역할을 할 것입니다.

이번 뉴스레터를 통해 최신 AI 트렌드를 살펴보았습니다. AI는 이제 단순한 도구가 아니라, 기업의 경쟁력을 강화하는 필수 요소로 자리 잡고 있습니다. 앞으로도 흥미로운 기술 소식을 전해드릴 예정이니, 많은 기대 부탁드립니다! 다음 호에서 만나요!